In [7]:
import datetime, json, subprocess, time, urllib2

In [8]:
key = json.load(open('bustime_api_key.json'))['bustime_api_key']

# BusTime Developer API version 3 guide
http://truetime.portauthority.org/bustime/apidoc/docs/DeveloperAPIGuide3_0.pdf
   
   
### 1.5 Is there a limit to the number of requests I can make to the Developer API?

Yes. By default, one API key can make a maximum of 10,000 requests per day. If you believe
that you will require more than 10,000 daily requests, you must request that the cap on your
key be raised to handle the additional traffic

In [9]:
def call_api(path, args=''):
    url = 'http://truetime.portauthority.org/bustime/api/%s?key=%s&format=json&%s' % (path, key, args)
    # use curl;  we can't use urllib2 because the bustime server requires HTTP/1.1 and returns 505 if we attempt 1.0
    return json.loads(subprocess.check_output(['/usr/bin/curl', url]))

In [10]:
#call_api('v3/getrtpidatafeeds')

In [11]:
routes = call_api('v3/getroutes')
route_ids = [r['rt'] for r in routes['bustime-response']['routes']]
', '.join(route_ids)

u'BLLB, BLSV, RED, 1, 11, 12, 13, 14, 15, 16, 17, 18, 19L, 2, 20, 21, 22, 24, 26, 27, 28X, 29, 31, 36, 38, 39, 4, 40, 41, 43, 44, 48, 51, 51L, 52L, 53, 53L, 54, 55, 56, 57, 58, 59, 6, 60, 61A, 61B, 61C, 61D, 64, 65, 67, 68, 69, 7, 71, 71A, 71B, 71C, 71D, 74, 75, 77, 79, 8, 81, 82, 83, 86, 87, 88, 89, 91, 93, G2, G3, G31, O1, O12, O5, P1, P10, P12, P13, P16, P17, P2, P3, P67, P68, P69, P7, P71, P76, P78, Y1, Y45, Y46, Y47, Y49'

In [ ]:
def log_all():
    t = time.time()
    ret = {}
    printable_timestamp = datetime.datetime.fromtimestamp(t).strftime('%Y%m%d %H:%M:%S')
    ret["timestamp_epoch"] = t
    ret["timestamp_printable"] = printable_timestamp
    ret["getvehicles"] = []
    for i in range(0, len(route_ids), 10):
        ret["getvehicles"].append(call_api('v3/getvehicles', 'tmres=s&rt=' + ','.join(route_ids[i:i+10])))
    filename = datetime.datetime.fromtimestamp(t).strftime('data/%Y%m%d.log')
    record = json.dumps(ret) + '\n'
    open(filename, 'a').write(record)
    print '%s: Appended %d bytes to %s' % (printable_timestamp, len(record), filename)
    
cadence = 90 # seconds
target_time = int(time.time() / cadence) * cadence
next_day = int(target_time / 86400 + 1) * 86400

original_date = datetime.datetime.fromtimestamp(target_time).date()

# Loop for a day
while target_time < next_day:
    t = time.time()
    if t < target_time:
        time.sleep(target_time - t)
    log_all()
    target_time += cadence

20180217 10:30:27: Appended 49706 bytes to data/20180217.log
